In [38]:
import vaex
import h3

df_taxi_trips = vaex.open('./data/trips_prepared.hdf5')
df_taxi_trips.head()

df_taxi_trips["trip_start_day"] = df_taxi_trips.trip_start_timestamp.dt.day
df_taxi_trips["trip_end_day"] = df_taxi_trips.trip_end_timestamp.dt.day
df_taxi_trips["trip_start_month"] = df_taxi_trips.trip_start_timestamp.dt.month
df_taxi_trips["trip_end_month"] = df_taxi_trips.trip_end_timestamp.dt.month

In [39]:
def geo_to_h3(row1, row2, resolution):
    return h3.geo_to_h3(row1,row2, resolution)

# Step 1: For each pickup and drop-off calculate the correct hexagon in the resolution
df_taxi_trips['pickup_hex'] = df_taxi_trips.apply(geo_to_h3, [df_taxi_trips['pickup_centroid_latitude'], df_taxi_trips['pickup_centroid_longitude']])
df_taxi_trips['dropoff_hex'] = df_taxi_trips.apply(geo_to_h3, [df_taxi_trips['dropoff_centroid_latitude'], df_taxi_trips['dropoff_centroid_longitude']])

In [37]:
grouped_df = df_taxi_trips.groupby(['trip_start_hour', 'trip_start_month', 'trip_start_day']).agg({'demand': 'count'})

In [13]:
# df_demand = df_taxi_trips.resample(rule='H', on='trip_start_hour').size().reset_index(name='demand')
# df_demand.columns = ['date_time', 'demand']
# df_demand.head()

df_taxi_trips.groupby(by='trip_start_hour')
df_taxi_trips

#,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,trip_start_hour,trip_end_hour,trip_start_4h_period,trip_end_4h_period
0,1b545d0be27b195076e5d2a7d4fc0ebef6ef20ac,'56a1119c6ca57e39525cf06829f9ecff553cf4b5ac24821...,2017-01-01 00:00:00.000000,2017-01-01 00:00:00.000000,240.0,1.3,17031839100.0,17031833000.0,32.0,28.0,6.25,0.0,0.0,0.0,6.25,Cash,Taxi Affiliation Services,41.880994471,-87.632746489,41.88528132,-87.6572332,0,0,0,0
1,c3ee40051c5643c54064a8b9df3686d2d411518d,'7987cfe5e3b2c9869665f70fe35dbe7423e7fc5852f2d6f...,2017-01-01 00:00:00.000000,2017-01-01 00:15:00.000000,1500.0,0.4,17031839100.0,17031081300.0,32.0,8.0,22.75,4.85,0.0,1.5,29.1,Credit Card,Blue Ribbon Taxi Association Inc.,41.880994471,-87.632746489,41.898331794,-87.620762865,0,0,0,0
2,29f30c8a6bbc028f5d63aa18c708d31b47267a9f,'0b269137bd94b55212311226d9e1554423e12bcc16a432c...,2017-01-01 00:00:00.000000,2017-01-01 00:30:00.000000,1620.0,0.9,17031839100.0,17031081500.0,32.0,8.0,12.25,2.85,0.0,2.0,17.1,Credit Card,Taxi Affiliation Services,41.880994471,-87.632746489,41.892507781,-87.626214906,0,0,0,0
3,55ae4fa955e6bc8705b995248c34f4183d848c27,'d1d781b6807176ed53fbb9d0010dd0a87e29c073984bda6...,2017-01-01 00:00:00.000000,2017-01-01 00:00:00.000000,540.0,1.0,17031081202.0,17031081403.0,8.0,8.0,6.75,0.0,0.0,2.0,8.75,Cash,Dispatch Taxi Affiliation,41.902788048,-87.62614559,41.890922026,-87.618868355,0,0,0,0
4,ce908cd441e28d08dfb22bf168102beabb5e16e4,'f7630edc68a131c4a97382dd93206949da80ec1443d8da7...,2017-01-01 00:00:00.000000,2017-01-01 00:15:00.000000,480.0,0.5,17031081500.0,17031081403.0,8.0,8.0,6.25,2.0,0.0,1.5,9.75,Credit Card,Taxi Affiliation Services,41.892507781,-87.626214906,41.890922026,-87.618868355,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"10,980,705",ee84ddee043ef4937e10000139b096c9eccd0dc9,'ca85563c7c46f07258ff388e8a43a1502115d1824dd00b5...,2017-12-31 23:45:00.000000,2017-12-31 23:45:00.000000,622.0,0.96,17031081700.0,17031081401.0,8.0,8.0,7.25,2.0,0.0,1.0,10.75,Credit Card,Taxi Affiliation Service Yellow,41.892042136,-87.63186395,41.89503345,-87.619710672,23,23,5,5
"10,980,706",1e3d7f719c6f360d070b4b02bb5a237fd70be799,'c4a594a1f522860a7f91aca9a3cccbef723a1178f1fde92...,2017-12-31 23:45:00.000000,2017-12-31 23:45:00.000000,300.0,1.2,17031081500.0,17031080100.0,8.0,8.0,6.25,0.0,0.0,0.0,6.25,Cash,Taxi Affiliation Service Yellow,41.892507781,-87.626214906,41.907520075,-87.6266589,23,23,5,5
"10,980,707",70fa927cfd759f5f9e6d0ca7606cb3b2112a6ca8,'111ced8a81014a3a549105d512397282b1a96f487a65c18...,2017-12-31 23:45:00.000000,2018-01-01 00:00:00.000000,644.0,1.01,17031081402.0,17031081300.0,8.0,8.0,7.25,0.0,0.0,1.0,8.25,Cash,Taxi Affiliation Service Yellow,41.891971508,-87.612945414,41.898331794,-87.620762865,23,0,5,0
"10,980,708",1a3504d474e1eee8d80283edbb2df4f20b18420d,'1b7aef0344f45914e7942efad05553934fe222c880a1cb1...,2017-12-31 23:45:00.000000,2017-12-31 23:45:00.000000,381.0,1.52,17031241400.0,17031842200.0,24.0,8.0,7.25,0.0,0.0,1.5,8.75,Cash,Nova Taxi Affiliation Llc,41.906025969,-87.675311622,41.904935302,-87.649907226,23,23,5,5


In [35]:
df_weather = pd.read_csv('./data/weather_data_final.csv')
df_weather['date_time'] = pd.to_datetime(df_weather['date_time'], format='%Y-%m-%d %X')

In [ ]:
df_weather = pd.read_csv('./data/weather_data_final.csv')
df_weather['date_time'] = pd.to_datetime(df_weather['date_time'], format='%Y-%m-%d %X')
df_demand = df_demand.merge(df_weather, how='left', on='date_time')

df_hourly_demand = df_demand.copy()
df_demand.head()

# Feature Engineering

### Previous demand as input

As we have given time series data, it is a common approach to use the demand of previous hours (or days etc.) as an input for the prediction. The assumption we hereby make is that the factors that influence the demand have not changed dramatically within the used time frames. We have decided to construct the following features from previous demand:

* 2 hour: The asssumption is that the demand should not change dramatically between three hours.
* 24 hours: The asssumption is that the current demand should be comparable to the demand exactly one day ago, as factors such as season, time of the day are the same.
* Average demand of the past week at the same day time: This feature is the average of all 7 demand observations of the past week at same time of the day. 

In [ ]:
df_taxi_trips['demand_h-2'] = df_taxi_trips.demand.shift(periods=2)
df_taxi_trips['demand_h-24'] = df_taxi_trips.demand.shift(periods=24)